In [1]:
import os
from glob import glob
import sys
sys.path.append(".")

from argparse import ArgumentParser
from typing import Dict
import json
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
def parse_args():
    parser = ArgumentParser()
    parser.add_argument(
        'inputs',
        type=str,
        nargs='?',
        default = './tests/data/kinect/color/',
        help='Input image/video path or folder path.')
    parser.add_argument(
        '--pose3d',
        type=str,
        default='human3d',
        help='Pretrained 3D pose estimation algorithm. It\'s the path to the '
        'config file or the model name defined in metafile.')
    parser.add_argument(
        '--pose3d-weights',
        type=str,
        default=None,
        help='Path to the custom checkpoint file of the selected pose model. '
        'If it is not specified and "pose3d" is a model name of metafile, '
        'the weights will be loaded from metafile.')
    parser.add_argument(
        '--device',
        type=str,
        default=None,
        help='Device used for inference. '
        'If not specified, the available device will be automatically used.')
    parser.add_argument(
        '--vis-out-dir',
        type=str,
        default='./vis_results/human3d',
        help='Directory for saving visualized results.')
    parser.add_argument(
        '--pred-out-dir',
        type=str,
        default='./vis_results/human3d',
        help='Directory for saving inference results.')
    parser.add_argument(
        '--show-alias',
        action='store_true',
        help='Display all the available model aliases.')
    
    call_args = vars(parser.parse_args())

    init_kws = [
        'device', 'pose3d', 'pose3d_weights'
    ]
    init_args = {}
    for init_kw in init_kws:
        init_args[init_kw] = call_args.pop(init_kw)

    diaplay_alias = call_args.pop('show_alias')

    return init_args, call_args, diaplay_alias

In [ ]:
LIMBS_KINECT = np.array([[20, 3], # head
                         [20, 8], [8, 9], [9, 10], # right arm
                         [20, 4], [4, 5], [5, 6], # left arm
                         [20, 16], [16, 17], [17, 18], # right leg
                         [20, 12], [12, 13], [13, 14]]) # left leg --> 왼쪽 다리 안나옴

In [ ]:
def display_model_aliases(model_aliases: Dict[str, str]) -> None:
    """Display the available model aliases and their corresponding model
    names."""
    aliases = list(model_aliases.keys())
    max_alias_length = max(map(len, aliases))
    print(f'{"ALIAS".ljust(max_alias_length+2)}MODEL_NAME')
    for alias in sorted(aliases):
        print(f'{alias.ljust(max_alias_length+2)}{model_aliases[alias]}')

# 이미지 파일명에서 프레임 번호 추출 (예: '1_00000200.png' -> 200)
def extract_frame_id(image_path):
    frame_id_match = re.search(r'(\d+)_(\d+).png', image_path)
    if frame_id_match:
        first_num = frame_id_match.group(1)
        second_num = str(int(frame_id_match.group(2)))  # 앞의 0들을 제거하여 숫자 형태로 변환
        return f"{first_num}_{second_num}"  # 첫 번째 숫자와 뒷부분의 숫자를 연결하여 반환
    else:
        print("Could not extract frame number from the image path")
        return None
    
# 추가된 함수: Kinect SDK 예측 결과를 로드하는 함수
def load_kinect_sdk_predictions(path_to_sdk_json, frame_id):
    with open(path_to_sdk_json, 'r') as file:
        pred_sdk = json.load(file)

    # 문자열 형식의 frame_id를 분리하고 숫자 부분만 정수로 변환
    frame_id_num = int(frame_id.split('_')[1])

    try:
        if frame_id_num < len(pred_sdk) and len(pred_sdk[frame_id_num]) > 0:
            print(f"Loaded prediction for frame index {frame_id_num}")
            return pred_sdk[frame_id_num]
        else:
            print(f"No prediction data found for frame index {frame_id_num}")
            return None
    except IndexError as e:
        print(f"Error accessing data for frame {frame_id_num}: {e}")
        return None
    
def load_gt(path_to_gt_json, frame_id):
    # JSON 파일로부터 GT 결과를 로드
    with open(path_to_gt_json, 'r') as file:
        anno_gt = json.load(file)

    # 문자열 형식의 frame_id를 분리하고 숫자 부분만 정수로 변환
    frame_id_num = int(frame_id.split('_')[1])

    # 리스트 내에서 인덱스를 기반으로 예측 결과 반환
    if 0 <= frame_id_num < len(anno_gt):
        print(f"Loaded annotation for frame index {frame_id_num}")
        return anno_gt[frame_id_num]
    else:
        print(f"No annotation data found for frame index {frame_id_num}")
        return None
    
def filter_kinect_joints(kinect_joints, joint_mapping):
    # Kinect SDK 조인트 좌표 배열에서 MMPose와 매핑된 조인트만 추출
    filtered_joints = [kinect_joints[joint_mapping[joint]['kinect']] for joint in joint_mapping]
    return np.array(filtered_joints)